In [1]:
import cv2
import numpy as np
import dlib

cap = cv2.VideoCapture(0)
font=cv2.FONT_HERSHEY_DUPLEX

face_detect = dlib.get_frontal_face_detector()
pred = dlib.shape_predictor("facepoints.dat")

def gaze_ratio(eyepoints, face_landmark_points, frame, gray):
    left_eye_points = np.array([(face_landmark_points.part(eyepoints[0]).x, face_landmark_points.part(eyepoints[0]).y),
                            (face_landmark_points.part(eyepoints[1]).x, face_landmark_points.part(eyepoints[1]).y),
                            (face_landmark_points.part(eyepoints[2]).x, face_landmark_points.part(eyepoints[2]).y),
                            (face_landmark_points.part(eyepoints[3]).x, face_landmark_points.part(eyepoints[3]).y),
                            (face_landmark_points.part(eyepoints[4]).x, face_landmark_points.part(eyepoints[4]).y),
                            (face_landmark_points.part(eyepoints[5]).x, face_landmark_points.part(eyepoints[5]).y)], np.int32)
    h, w, _ = frame.shape
    mask = np.zeros((h, w), np.uint8)
    cv2.polylines(mask, [left_eye_points], True, 255, 2)
    cv2.fillPoly(mask, [left_eye_points], 255)
    left_eye = cv2.bitwise_and(gray, mask, mask=mask)
    xmin = np.min(left_eye_points[:, 0])
    xmax = np.max(left_eye_points[:, 0])
    ymin = np.min(left_eye_points[:, 1])
    ymax = np.max(left_eye_points[:, 1])
    gray_eye = left_eye[ymin: ymax, xmin: xmax]
    _, thresholded_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    thresholded_eye = cv2.resize(thresholded_eye, None, fx=5, fy=5)
    h, w = thresholded_eye.shape
    left_side_threshold = thresholded_eye[0: h, 0: int(w / 2)]
    white_part_in_left = cv2.countNonZero(left_side_threshold)
    right_side_threshold = thresholded_eye[0: h, int(w / 2): w]
    white_part_in_right = cv2.countNonZero(right_side_threshold)
    if white_part_in_right==0:
        gaze_ratio=0.5
    else:     
        gaze_ratio = white_part_in_left / white_part_in_right
    return left_eye, gaze_ratio


def masky(eyepoints, face_landmark_points, frame):
    left_eye_points = np.array([(face_landmark_points.part(eyepoints[0]).x, face_landmark_points.part(eyepoints[0]).y),
                            (face_landmark_points.part(eyepoints[1]).x, face_landmark_points.part(eyepoints[1]).y),
                            (face_landmark_points.part(eyepoints[2]).x, face_landmark_points.part(eyepoints[2]).y),
                            (face_landmark_points.part(eyepoints[3]).x, face_landmark_points.part(eyepoints[3]).y),
                            (face_landmark_points.part(eyepoints[4]).x, face_landmark_points.part(eyepoints[4]).y),
                            (face_landmark_points.part(eyepoints[5]).x, face_landmark_points.part(eyepoints[5]).y)], np.int32)
    h, w, _ = frame.shape
    mask = 255*np.ones((h, w), np.uint8)
    cv2.polylines(mask, [left_eye_points], True, 7, 2)
    cv2.fillPoly(mask, [left_eye_points], 7)
    return mask


while True:
    ret, frame = cap.read()
    if ret is True: 
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detect(frame)
        for f in faces:
            x1, y1 = f.left(), f.top()
            x2, y2 = f.right(), f.bottom()
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
            face_landmark_points = pred(gray, f)
            h, w, _ = frame.shape
            mask = 255*np.ones((h, w), np.uint8)
            #m=np.ones((height, width), np.uint8)
            #mask1 = np.zeros((height, width), np.uint8)
        # gaze detection
            left, lefteye_gaze_ratio =gaze_ratio([36, 37, 38, 39, 40, 41], face_landmark_points, frame, gray)
            right, righteye_gaze_ratio = gaze_ratio([42, 43, 44, 45, 46, 47], face_landmark_points, frame, gray)
            gaze_ratio = (righteye_gaze_ratio + lefteye_gaze_ratio) / 2
            
            if gaze_ratio <= 1:
                cv2.putText(frame, "Right", (50, 100), font, 2, (0, 0, 255), 3)
            elif 1 < gaze_ratio < 1.7:
                cv2.putText(frame, "Center", (50, 100), font, 2, (0, 255, 0), 3)
            else:
                cv2.putText(frame, "Left", (50, 100), font, 2, (255, 0, 0), 3)
              
            
            mask=cv2.bitwise_or(left, right, mask=None)
            m_left=masky([36, 37, 38, 39, 40, 41], face_landmark_points, frame)
            m_right=masky([42, 43, 44, 45, 46, 47], face_landmark_points, frame)
            m=cv2.bitwise_and(m_left, m_right, mask=None)
            
            mimp=cv2.bitwise_or(m, mask, mask=None)
            _, k=cv2.threshold(mimp,35, 255, cv2.THRESH_BINARY )  
            
            _, contours, _=cv2.findContours(k, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            print(contours)
            for cnt in contours:
                cv2.drawContours(frame, cnt, -1, (0, 255, 0), 3 )
                
            
            cv2.imshow('main2', k)
            #cv2.imshow('left', left)
            cv2.imshow('main', mimp)
            #cv2.imshow('2eyesopp', m)
            cv2.imshow('2eyes', mask)
            #cv2.imshow('lefteye', k)
            
        cv2.imshow('frame',frame)

    k=cv2.waitKey(1) & 0xFF
    if k==ord('m'):
        break

cap.release()
cv2.destroyAllWindows()

[array([[[  0,   0]],

       [[  0, 479]],

       [[639, 479]],

       [[639,   0]]], dtype=int32), array([[[425, 294]],

       [[426, 293]],

       [[427, 293]],

       [[428, 294]],

       [[427, 295]],

       [[426, 295]]], dtype=int32), array([[[420, 293]],

       [[421, 292]],

       [[422, 292]],

       [[423, 293]],

       [[423, 296]],

       [[422, 297]],

       [[423, 296]],

       [[425, 298]],

       [[427, 298]],

       [[428, 299]],

       [[432, 299]],

       [[433, 298]],

       [[432, 297]],

       [[433, 296]],

       [[431, 296]],

       [[430, 295]],

       [[429, 295]],

       [[428, 294]],

       [[429, 293]],

       [[431, 293]],

       [[432, 292]],

       [[435, 292]],

       [[437, 294]],

       [[438, 294]],

       [[440, 296]],

       [[441, 296]],

       [[443, 298]],

       [[444, 298]],

       [[445, 299]],

       [[445, 300]],

       [[443, 302]],

       [[442, 302]],

       [[441, 303]],

       [[438, 303]],

   

TypeError: 'float' object is not callable